In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json
import warnings
from tqdm import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# 2020년대 트레이드 내역 데이터 불러오기
with open("player_moves/trade_list_2020s.json") as f:
    trades = json.load(f)

# 2020년데 지명권 트레이드 결과 데이터 불러오기
draft_tickets = pd.read_csv("player_moves/draft_tickets_2020s.csv", keep_default_na=False)

In [3]:
def find_team(input_string):
    substrings = ['KIA', '해태', '삼성', 'OB', '두산', 'SK', 'SSG', '삼미', '청보', '태평양', '현대',
        'MBC', 'LG', '롯데', '빙그레', '한화', 'NC', 'KT', '넥센', '키움', '히어로즈', '쌍방울']
    for substring in substrings:
        if substring in input_string:
            return substring
    return None 

In [4]:
def new_team_name(team):
    if team == '해태':
        return 'KIA'
    elif team == 'OB':
        return '두산'
    elif team == 'SK':
        return 'SSG'
    elif team in ['삼미', '청보', '태평양']:
        return '현대'
    elif team == 'MBC':
        return 'LG'
    elif team == '빙그레':
        return '한화'
    elif team in ['히어로즈', '넥센']:
        return '키움'
    else:
        return team

#### 2020년대 트레이드된 선수들의 팀 이동 기록 크롤링

In [5]:
for trade in tqdm(trades):
    for x in trade["playerA"] + trade["playerB"]:
        if x["type"] == "money":
            continue

        elif x["type"] == "draft":
            player_id = draft_tickets.loc[(draft_tickets["id"] == trade["id"]) & \
                + (draft_tickets["지명라운드"] == x["round"]), "statizId"].item()
            name = draft_tickets.loc[(draft_tickets["id"] == trade["id"]) & \
                + (draft_tickets["지명라운드"] == x["round"]), "선수"].item()
            
            if player_id == '':
                continue

        else:
            name = x["name"]
            player_id = x["statizId"]

        player_url = f"https://statiz.sporki.com/player/?m=playerinfo&p_no={player_id}"
        driver = webdriver.Chrome()
        driver.get(player_url)
        time.sleep(0.5)

        rows = driver.find_elements(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.box_type_boared > div:nth-child(5) > div > div > div.box_cont > div > table > tbody > tr")
        move_list = []
        for index, row in enumerate(rows):
            td = list(map(lambda x : x.text, row.find_elements(By.CSS_SELECTOR, "tr > td")))
            if index == 0:
                move_list.append({"start_date" : str(td[0])[:4] + "-01-01", 
                                    "team" : new_team_name(find_team(str(td[2])))})

            else:
                team = new_team_name(td[2][td[2].find(">")+2:td[2].find("/")-1])
                if team == '':
                    team = new_team_name(td[2][td[2].find(">")+2:])

                if move_list[-1]["team"] == new_team_name(team):
                    continue
                else:
                    move_list.append({"start_date" : td[2][:td[2].find(",")], 
                                        "team" : new_team_name(team)})

        move_json = {
            "name" : name,
            "statizId" : player_id,
            "team_history" : move_list
        }

        driver.quit()

        with open(f"player_moves/{name}_{player_id}_team_history.json", 'w', encoding='UTF-8-sig') as file:
            file.write(json.dumps(move_json, ensure_ascii=False, indent=2))

  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [16:06<00:00, 24.16s/it]
